In [4]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
import datetime
import json
# Import API key
from config import weather_api_key

### Get Long Lat list of tuples to pass into Weather API

In [5]:
geojson_data = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.geojson'

In [23]:
response = requests.get(geojson_data).json()
features = response["features"]
test = features[0]['properties']
coords_list = []
for data in features:
    time = data['properties']['time']
    long_coords = data["geometry"]["coordinates"][0]
    lat_coords = data["geometry"]["coordinates"][1]
    coords = {
            lat_coords,
            long_coords,
            time }
    
    coords_list.append(coords)
print(coords_list)
# coords_list

[{-175.5813, -23.9757, 1597189361979}, {24.1466, 1597187246001, 122.7857}, {1597181248998, 94.8302, 30.3671}, {94.7675, 1597169659849, 30.3786}, {16.7661, 1597161569595, -85.9341}, {-176.7117, 1597143221834, -28.1755}, {1597132571384, 2.5148, 126.8777}, {1597130801530, 11.9395, 124.2146}, {1597127508369, 51.9797, 28.6581}, {-23.9032, -66.702, 1597125099119}, {105.121, 1597124519283, 28.9031}, {40.832, 1597122619690, -13.9712}, {1597101914155, -12.371, -3.901}, {161.3113, 1597099146993, -10.4049}, {33.2468333, 1597074974330, -115.69}, {178.7358, 51.3351, 1597069821455}, {16.8514, 1597068807353, -85.8675}, {1597061585633, -4.7616, 143.5143}, {-71.7941, 1597050915107, -33.48}, {1597019080026, 35.3743, 140.535}, {11.6696, -53.0485, 1597016553419}, {-142.8012, 60.5147, 1597014650110}, {1597012807908, 51.1302, 156.412}, {1597012604435, -3.2673, 143.749}, {1597011141346, 110.2577, -9.3536}, {1597008677081, 119.0197, -9.815}, {1597005628185, 2.6893, -84.4819}, {94.865, 1596993213580, 30.4371},

In [77]:
test = list(coords_list[0])[2]
test

1597189361979

In [76]:
dt3 = datetime.datetime.fromtimestamp(test / 1000)
dt3


datetime.datetime(2020, 8, 11, 16, 42, 41, 979000)

In [74]:

# Time range end is the day of the earthquake
time_range_end = str(dt3.year) + "/" + str(dt3.month) + "/" + str(dt3.day)
time_range_end

'2020/8/11'

In [75]:
# Time range begin is the day of the earthquake
three_days_before = (dt3.day) - 3

time_range_begin = str(dt3.year) + "/" + str(dt3.month) + "/" + str(three_days_before)
time_range_begin

'2020/8/8'

In [25]:
len(coords_list)

485

-----

### Use Coordinates to get historical weather data.
Coordinates represent earthquake lat long location

In [8]:
from pprint import pprint

In [24]:
tuple_of_data = []
add_tuple = {}
counter = 0
for coordinates in coords_list:
    base_url = 'http://api.weatherstack.com/historical'
    params_weather = {'access_key': weather_api_key, 
                      'query': coordinates,
                      'historical_date_start': time_range_begin,
                        'historical_date_end': time_range_end
                 }
    response = requests.get(base_url, params=params_weather).json()
    try: 
        # extract results
        location = response.get("location")

        # City, Country, Region
        city = location["name"]
        country = location["country"]
        region = location["region"]

        # Historical Only
        historical = response['historical']

        for hist in historical:
            add_tuple[hist] = {
                'city': location.get('name'),
                'country': location["country"],
                'region': location["region"],
                'avgtemp': historical[hist]['avgtemp'],
                'date': historical[hist]['avgtemp'],
                'date_epoch': historical[hist]['avgtemp'],
                'maxtemp': historical[hist]['avgtemp'],
                'mintemp': historical[hist]['avgtemp'],
                'sunhour': historical[hist]['avgtemp'],
                'totalsnow': historical[hist]['avgtemp'],
                'uv_index': historical[hist]['avgtemp'],
                'long': coordinates[0],
                'lat': coordinates[1]
            }
        tuple_of_data.append(add_tuple)
#         pprint(tuple_of_data)
    except:
        pass
    counter += 1  
    print(counter)
    print(coordinates[2])
    break


TypeError: 'set' object does not support indexing